## Section I

Operations on Spark RDDs

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [3]:
df_green = spark.read.parquet('D:/data/pq/green/*/*')

#### We will be Implementing the below SQL code using RDDs

SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone, 
    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records 
FROM
    green \
WHERE 
    lpep_pickup_datetime >= '2020-01-01 00:00:00' \
GROUP BY
    1, 2;

In [4]:
# STEP 1

rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [5]:
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), PULocationID=67, total_amount=33.45),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41), PULocationID=260, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 16, 32, 26), PULocationID=82, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 22, 42), PULocationID=166, total_amount=12.74)]

In [6]:
# STEP 2

from datetime import datetime

start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

rdd.filter(filter_outliers).take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), PULocationID=67, total_amount=33.45),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41), PULocationID=260, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 16, 32, 26), PULocationID=82, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 22, 42), PULocationID=166, total_amount=12.74)]

In [7]:
# Used this row for testing purpose

rows = rdd.take(10)
row = rows[0]

row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97)

In [8]:
# STEP 3

def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [9]:
rdd.filter(filter_outliers).map(prepare_for_grouping).take(5)

[((datetime.datetime(2020, 1, 23, 13, 0), 74), (44.97, 1)),
 ((datetime.datetime(2020, 1, 20, 15, 0), 67), (33.45, 1)),
 ((datetime.datetime(2020, 1, 15, 20, 0), 260), (8.3, 1)),
 ((datetime.datetime(2020, 1, 5, 16, 0), 82), (8.3, 1)),
 ((datetime.datetime(2020, 1, 29, 19, 0), 166), (12.74, 1))]

In [10]:
# STEP 4

def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

rdd.filter(filter_outliers).map(prepare_for_grouping).reduceByKey(calculate_revenue).take(5)

[((datetime.datetime(2020, 1, 20, 15, 0), 67), (79.5, 3)),
 ((datetime.datetime(2020, 1, 16, 8, 0), 41), (736.1399999999994, 54)),
 ((datetime.datetime(2020, 1, 20, 15, 0), 75), (609.0, 47)),
 ((datetime.datetime(2020, 1, 17, 21, 0), 74), (594.87, 39)),
 ((datetime.datetime(2020, 1, 3, 9, 0), 61), (142.21, 9))]

In [11]:
# STEP 5

from collections import namedtuple

RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

rdd.filter(filter_outliers).map(prepare_for_grouping).reduceByKey(calculate_revenue).map(unwrap).take(5)

[RevenueRow(hour=datetime.datetime(2020, 1, 20, 15, 0), zone=67, revenue=79.5, count=3),
 RevenueRow(hour=datetime.datetime(2020, 1, 16, 8, 0), zone=41, revenue=736.1399999999994, count=54),
 RevenueRow(hour=datetime.datetime(2020, 1, 20, 15, 0), zone=75, revenue=609.0, count=47),
 RevenueRow(hour=datetime.datetime(2020, 1, 17, 21, 0), zone=74, revenue=594.87, count=39),
 RevenueRow(hour=datetime.datetime(2020, 1, 3, 9, 0), zone=61, revenue=142.21, count=9)]

In [18]:
# If you don't specify the schema in toDF(), it takes a bit longer to run as it tries to find what schema to implement it in

rdd.filter(filter_outliers).map(prepare_for_grouping).reduceByKey(calculate_revenue).map(unwrap).toDF().show(5)

+-------------------+----+-----------------+-----+
|               hour|zone|          revenue|count|
+-------------------+----+-----------------+-----+
|2020-01-20 15:00:00|  67|             79.5|    3|
|2020-01-16 08:00:00|  41|736.1399999999994|   54|
|2020-01-20 15:00:00|  75|            609.0|   47|
|2020-01-17 21:00:00|  74|           594.87|   39|
|2020-01-03 09:00:00|  61|           142.21|    9|
+-------------------+----+-----------------+-----+
only showing top 5 rows



In [19]:
# STEP 6

from pyspark.sql import types

result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

rdd.filter(filter_outliers).map(prepare_for_grouping).reduceByKey(calculate_revenue).map(unwrap).toDF(result_schema).show(5)

+-------------------+----+-----------------+-----+
|               hour|zone|          revenue|count|
+-------------------+----+-----------------+-----+
|2020-01-20 15:00:00|  67|             79.5|    3|
|2020-01-16 08:00:00|  41|736.1399999999994|   54|
|2020-01-20 15:00:00|  75|            609.0|   47|
|2020-01-17 21:00:00|  74|           594.87|   39|
|2020-01-03 09:00:00|  61|           142.21|    9|
+-------------------+----+-----------------+-----+
only showing top 5 rows



In [20]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [15]:
df_result.write.parquet('D:/tmp/green-revenue')

## Section II

Spark RDD mapPartition

In [21]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [23]:
duration_rdd.take(5)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, DOLocationID=130, trip_distance=12.77),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), PULocationID=67, DOLocationID=39, trip_distance=8.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41), PULocationID=260, DOLocationID=157, trip_distance=1.27),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 16, 32, 26), PULocationID=82, DOLocationID=83, trip_distance=1.25),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 22, 42), PULocationID=166, DOLocationID=42, trip_distance=1.84)]

In [29]:
import pandas as pd

#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred


def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [31]:
duration_rdd.mapPartitions(apply_model_in_batch).take(5)

[Pandas(Index=0, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-23 13:10:15'), PULocationID=74, DOLocationID=130, trip_distance=12.77, predicted_duration=63.849999999999994),
 Pandas(Index=1, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-20 15:09:00'), PULocationID=67, DOLocationID=39, trip_distance=8.0, predicted_duration=40.0),
 Pandas(Index=2, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-15 20:23:41'), PULocationID=260, DOLocationID=157, trip_distance=1.27, predicted_duration=6.35),
 Pandas(Index=3, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-05 16:32:26'), PULocationID=82, DOLocationID=83, trip_distance=1.25, predicted_duration=6.25),
 Pandas(Index=4, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-29 19:22:42'), PULocationID=166, DOLocationID=42, trip_distance=1.84, predicted_duration=9.200000000000001)]

In [32]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [33]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|63.849999999999994|
|              40.0|
|              6.35|
|              6.25|
| 9.200000000000001|
|               3.8|
|16.599999999999998|
|             11.05|
|               4.5|
|              30.5|
|               8.7|
|5.8999999999999995|
|              11.0|
|              15.2|
|              4.25|
|25.299999999999997|
|7.8500000000000005|
|              34.0|
| 5.300000000000001|
|              6.15|
+------------------+
only showing top 20 rows

